# Analyse préliminaire des données de Navigo

Nous vous proposons un ensemble de visualisations permettant d'avoir un aperçu sommaire des pointcalls et des flux associés au datasprint, ainsi que des produits ou activités mentionnés pour les voyages. Les visualisations visent notamment à donner un premier aperçu de la dimension d'incertitude encodée dans les données.

Nous avons également construit plusieurs visualisations permettant d'observer ces données en utilisant les systèmes de classifications de toflit18. 

Notes :

* dans cette analyse préliminaire on se fonde principalement sur les *pointcalls* et non les flux des bateaux (excepté la dernière partie de la page). Quand on raisonne en pointcalls, les objets manipulés sont donc des indications d'escales entrantes ("In") ou sortantes ("Out"), avec des niveaux d'incertitude différents (pour les développeuses et développeurs : les flux sont par ailleurs accessibles dans l'API via `portic_client.get_flows(source_subset='Poitou_1789)`)
* on compte parfois en pointcalls et parfois en nombre de `commodity_purposes` : ces métriques nous informent davantage sur la présence de données pour les objets étudiées que sur les quantités/valeurs effectives : il faut garder cette dimension en tête pour les interpréter correctement, et un travail d'affinage sera nécessaire à propos de cela pendant le datasprint.

In [ ]:
from poitousprint import Portic, get_pointcalls_port_as_toflit_partner, nest_portic_pointcall, get_pointcalls_commodity_purposes_as_toflit_product
import pandas as pds
from vega import VegaLite
from keplergl import KeplerGl

kepler_base_conf = {
    'version': 'v1',
    'config': {
        # centering the map on the region
        'mapState': {
            'latitude': 45.6876849,
            'longitude': -1.15,
            'zoom': 5.2
        }
    }
}

In [ ]:
portic_client = Portic()
pointcalls = portic_client.get_pointcalls(source_subset='Poitou_1789', year=1789)

uncertainty_annotations = {
    '0': '0 (confirmé)',
    '-1': '-1 (non confirmé)',
    '-2': '-2 (incertain car déduction)',
    '-3': '-3 (faux/incohérent)',
    '-4': '-4 (info manquante)'
}

def annotate_uncertainty(pointcall):
    p = pointcall.copy()
    p['pointcall_uncertainity'] = uncertainty_annotations[str(p['pointcall_uncertainity'])]
    return p
pointcalls = [annotate_uncertainty(p) for p in pointcalls]

## Aperçu des données sur les ports de la région

### Carte des pointcalls concernés par le sprint

In [ ]:
ports = {}
for p in pointcalls:
    port = p['pointcall']
    action = p['pointcall_action'].lower()
    in_region = p['pointcall_admiralty'] in ['Marennes', 'La Rochelle', "Sables-d’Olonne"]
    if port not in ports:
        lat = float(p['latitude']) if p['latitude'] is not None else None
        lon = float(p['longitude']) if p['longitude'] is not None else None
        ports[port] = {
            'in_region': 'amirautés de Marennes, La Rochelle et Sables d\'Olonne' if in_region else p['state_1789_fr'],
            'nb_pointcalls': 1,
            'latitude': lat,
            'longitude': lon,
            'ferme_bureau': p['ferme_bureau'],
            'etat': p['state_1789_fr']
        }
        ports[port][action] = 1
    else:
        ports[port]['nb_pointcalls'] += 1
        if action in ports[port]:
            ports[port][action] += 1
        else:
            ports[port][action] = 1
ports = [{'port': port, **vals} for port, vals in ports.items()]

points_map = KeplerGl(config={
  "version": "v1",
  "config": {
    "visState": {
      "filters": [],
      "layers": [
        {
          "id": "6e7g5tt",
          "type": "point",
          "config": {
            "dataId": "data_1",
            "label": "Point",
            "color": [
              241,
              92,
              23
            ],
            "columns": {
              "lat": "latitude",
              "lng": "longitude",
              "altitude": None
            },
            "isVisible": True,
            "visConfig": {
              "radius": 10,
              "fixedRadius": False,
              "opacity": 0.8,
              "outline": False,
              "thickness": 2,
              "strokeColor": None,
              "colorRange": {
                "name": "Uber Viz Qualitative 1.2",
                "type": "qualitative",
                "category": "Uber",
                "colors": [
                  "#12939A",
                  "#DDB27C",
                  "#88572C",
                  "#FF991F",
                  "#F15C17",
                  "#223F9A"
                ]
              },
              "strokeColorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "radiusRange": [
                1,
                20
              ],
              "filled": True
            },
            "hidden": False,
            "textLabel": [
              {
                "field": None,
                "color": [
                  255,
                  255,
                  255
                ],
                "size": 10,
                "offset": [
                  0,
                  0
                ],
                "anchor": "start",
                "alignment": "center"
              }
            ]
          },
          "visualChannels": {
            "colorField": {
              "name": "in_region",
              "type": "string"
            },
            "colorScale": "ordinal",
            "strokeColorField": None,
            "strokeColorScale": "quantile",
            "sizeField": {
              "name": "nb_pointcalls",
              "type": "integer"
            },
            "sizeScale": "sqrt"
          }
        }
      ],
      "interactionConfig": {
        "tooltip": {
          "fieldsToShow": {
            "data_1": [
              {
                "name": "port",
                "format": None
              },
              {
                "name": "in_region",
                "format": None
              },
              {
                "name": "nb_pointcalls",
                "format": None
              },
              {
                "name": "ferme_bureau",
                "format": None
              },
              {
                "name": "etat",
                "format": None
              }
            ]
          },
          "compareMode": False,
          "compareType": "absolute",
          "enabled": True
        },
        "brush": {
          "size": 0.5,
          "enabled": False
        },
        "geocoder": {
          "enabled": False
        },
        "coordinate": {
          "enabled": False
        }
      },
      "layerBlending": "normal",
      "splitMaps": [],
      "animationConfig": {
        "currentTime": None,
        "speed": 1
      }
    },
    "mapState": {
      "bearing": 0,
      "dragRotate": False,
      "latitude": 45.39153346341695,
      "longitude": -1.155246746228874,
      "pitch": 0,
      "zoom": 4.451272741107292,
      "isSplit": False
    },
    "mapStyle": {
      "styleType": "light",
      "topLayerGroups": {
        "label": False,
        "building": False,
        "water": True,
        "land": False
      },
      "visibleLayerGroups": {
        "label": True,
        "road": True,
        "border": False,
        "building": True,
        "water": True,
        "land": True,
        "3d building": False
      },
      "threeDBuildingColor": [
        9.665468314072013,
        17.18305478057247,
        31.1442867897876
      ],
      "mapStyles": {}
    }
  }
}, data={'data_1': pds.DataFrame(ports)})
points_map

<img src="dumps/visualisations/navigo/navigo_pointcalls_kepler.png">

### Légende

En rouge : pointcalls enregistrés dans la région de La Rochelle 

En bleu : les autres

### Carte des ports navigo et bureaux des Fermes Toflit concernés par le sprint

In [ ]:
from poitousprint import Portic, Toflit, get_pointcalls_commodity_purposes_as_toflit_product
portic_client = Portic()
toflit_client = Toflit()

# récupération de tous les ports navigo concernés

pointcalls_datasprint = portic_client.get_pointcalls(
 year = 1789,
 pointcall_admiralty = ['La Rochelle', 'Marennes', 'Sables d\'Olonne']
    # avec source_subset on otbient des ports hors de la région (car ils ont des flux en relation avec ceux de la région)
)

ports_navigo_dict = {} 
for pointcall in pointcalls_datasprint:
    is_bureau = False

    # si le port n'est pas encore enregistré dans les valeurs existantes on initialise les valeurs associées
    if pointcall['pointcall'] not in ports_navigo_dict.keys():
        if pointcall['pointcall'] == pointcall['ferme_bureau'] or pointcall['pointcall'] == 'Aligre de Marans' or pointcall['pointcall'] == 'Saint Martin de Ré': # ne marche pas pour certains noms ... (--> 'Aligre' ≠ 'Aligre de Marrans')
            is_bureau = True 
        
        # remplacer par "pas de bureau rattaché" plutot que None quand le port Navigo n'est pas rattaché à un bureau des Fermes Toflit18
        bureau = pointcall['ferme_bureau']
        if bureau is None:
            bureau = "pas de bureau rattaché"
            
        ports_navigo_dict[pointcall['pointcall']] = {
            'latitude':pointcall['latitude'],
            'longitude':pointcall['longitude'],
            'bureau':bureau,
            'is_bureau':is_bureau,
            'pointcall_count':1
            }
    
    #  si le port est déjà enregistré dans les valeurs existantes on incrémente seulement le compteur du nombre de pointcalls 
    else:
        ports_navigo_dict[pointcall['pointcall']]['pointcall_count'] += 1

In [ ]:
# Vérification : récupération des bureaux des Fermes concernés chez Toflit
flows = toflit_client.get_flows(
    year=1789, 
    customs_region='La Rochelle',
)

toflit_customs_offices_for_datasprint = set()
for flow in flows:
    toflit_customs_offices_for_datasprint.add(flow['customs_office'])

In [ ]:
# écriture dans un csv
import csv
from csv import DictWriter

# tableau qui contient une colonne par clé de dict 
with open('dumps/croisement_ports_bureaux_datasprint_aggregated.csv', 'w', newline='') as csvfile:
        fieldnames = ['pointcall','latitude', 'longitude', 'bureau', 'is_bureau', 'pointcall_count']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for port, values in ports_navigo_dict.items():
                    
            writer.writerow({
                'pointcall': port,
                'latitude': values['latitude'],
                'longitude': values['longitude'],
                'bureau': values['bureau'],
                'is_bureau': values['is_bureau'],
                'pointcall_count': values['pointcall_count']
            })

#### fichier csv disponible sur le git : https://github.com/medialab/portic-datasprint-2021/blob/main/preliminary_inquiry/dumps/croisement_ports_bureaux_datasprint_aggregated.csv

utilisable pour customiser les cartes existantes / générer d'autres cartes

In [ ]:
# Load an empty map
from keplergl import KeplerGl
from poitousprint import Portic
import pandas as pd

portic_client = Portic()

In [ ]:
# setting kepler config
config = {
    'version': 'v1',
    'config': {
        # centering the map on the region
        'mapState': {
            'latitude': 45.6876849,
            'longitude': -1.15,
            'zoom': 5.2
        }
    }
}

df = pd.read_csv('dumps/croisement_ports_bureaux_datasprint_aggregated.csv')
# reste à faire les réglages de couleur, proportionalité, mise en avant des bureaux ...

flux_map = KeplerGl(config=config, height=800, data={'data_1': df})
flux_map

In [ ]:
## Config intéressante

config = {
  "version": "v1",
  "config": {
    "visState": {
      "filters": [],
      "layers": [
        {
          "id": "5fl7ry",
          "type": "point",
          "config": {
            "dataId": "data_1",
            "label": "Point",
            "color": [
              255,
              203,
              153
            ],
            "columns": {
              "lat": "latitude",
              "lng": "longitude",
              "altitude": None
            },
            "isVisible": True,
            "visConfig": {
              "radius": 16.7,
              "fixedRadius": False,
              "opacity": 0.8,
              "outline": True,
              "thickness": 2,
              "strokeColor": None,
              "colorRange": {
                "name": "Custom Palette",
                "type": "custom",
                "category": "Custom",
                "colors": [
                  "#e41a1c",
                  "#377eb8",
                  "#4daf4a",
                  "#984ea3",
                  "#ff7f00",
                  "#ffff33",
                  "#a65628"
                ]
              },
              "strokeColorRange": {
                "name": "Custom Palette",
                "type": "custom",
                "category": "Custom",
                "colors": [
                  "#000000",
                  "#FFFFFF"
                ]
              },
              "radiusRange": [
                12.7,
                50
              ],
              "filled": True
            },
            "hidden": False,
            "textLabel": []
          },
          "visualChannels": {
            "colorField": {
              "name": "bureau",
              "type": "string"
            },
            "colorScale": "ordinal",
            "strokeColorField": {
              "name": "is_bureau",
              "type": "boolean"
            },
            "strokeColorScale": "ordinal",
            "sizeField": {
              "name": "pointcall_count",
              "type": "integer"
            },
            "sizeScale": "sqrt"
          }
        }
      ],
      "interactionConfig": {
        "tooltip": {
          "fieldsToShow": {
            "data_1": [
              {
                "name": "pointcall",
                "format": None
              },
              {
                "name": "bureau",
                "format": None
              },
              {
                "name": "is_bureau",
                "format": None
              },
              {
                "name": "pointcall_count",
                "format": None
              }
            ]
          },
          "compareMode": False,
          "compareType": "absolute",
          "enabled": True
        },
        "brush": {
          "size": 9.4,
          "enabled": False
        },
        "geocoder": {
          "enabled": False
        },
        "coordinate": {
          "enabled": False
        }
      },
      "layerBlending": "normal",
      "splitMaps": [],
      "animationConfig": {
        "currentTime": None,
        "speed": 1
      }
    },
    "mapState": {
      "bearing": 0,
      "dragRotate": False,
      "latitude": 45.94994663229237,
      "longitude": -0.9635546220911514,
      "pitch": 0,
      "zoom": 8.5, #7.469414939418524,
      "isSplit": False
    },
    "mapStyle": {
      "styleType": "dark",
      "topLayerGroups": {},
      "visibleLayerGroups": {
        "label": True,
        "road": True,
        "border": False,
        "building": True,
        "water": True,
        "land": True,
        "3d building": False
      },
      "threeDBuildingColor": [
        9.665468314072013,
        17.18305478057247,
        31.1442867897876
      ],
      "mapStyles": {}
    }
  }
}

flux_map = KeplerGl(config=config, height=800, data={'data_1': df})

"""
- légende particulière quand un port est un bureau des Fermes
- colorer de la même manière tous les ports rattachés à un même bureau des Fermes
- plus un port a de pointcalls disponible, plus il apparait en gros sur la carte

=> réalisation sur Kepler et Khartis pour comparer ce qui est possible de faire avec des 2 outils de cartographie

"""
flux_map

#### Carte obtenue avec Kepler

croisement ports Portic : bureaux des Fermes Navigo - datasprint PORTIC 2021/Ports de la région Poitou, Aunis, Saintonge en 1789 - Kepler
<img src="dumps/visualisations/croisement ports Portic : bureaux des Fermes Navigo - datasprint PORTIC 2021/ports_1789_Kepler.png">

#### Carte obtenue avec Khartis

(plus simple de gérer les labels)

In [ ]:
from IPython.core.display import SVG
SVG(filename='dumps/visualisations/croisement ports Portic : bureaux des Fermes Navigo - datasprint PORTIC 2021/Ports de la région Poitou, Aunis, Saintonge en 1789.svg')

In [ ]:
# Lister les ports des 3 amirautés
local_pointcalls = [pointcall for pointcall in pointcalls if pointcall['pointcall_admiralty'] in ['Marennes', 'La Rochelle', "Sables-d’Olonne"]]

for p in local_pointcalls:
    p['pointcall_action'] = p['pointcall_action'].lower()

VegaLite({
    "title": "Distribution des pointcalls de la région étudiée, par port et type",
    "width": 800,
    "mark": "bar",
    "encoding": {
        "x": {
            "type": "quantitative", 
            "aggregate": "count",
            "title": "nombre de pointcalls"
        },
        "y": {
            "field": "pointcall",
            "type": "nominal",
            "title": "port",
            "sort": "-x"
        },
        "color": {
            "type": "nominal",
            "field": "pointcall_action",
            "title": "type de pointcall ('pointcall_action')"
        }
    }
}, pds.DataFrame(local_pointcalls))

In [ ]:
# Lister les ports hors des 3 amirautés
not_local_pointcalls = [pointcall for pointcall in pointcalls if pointcall['pointcall_admiralty'] not in ['Marennes', 'La Rochelle', "Sables-d’Olonne"]]

for p in not_local_pointcalls:
    p['pointcall_action'] = p['pointcall_action'].lower()

VegaLite({
    "title": "Distribution des pointcalls hors région, par port et type",
    "width": 800,
    "mark": "bar",
    "encoding": {
        "x": {
            "type": "quantitative", 
            "aggregate": "count",
            "title": "nombre de pointcalls"
        },
        "y": {
            "field": "pointcall",
            "type": "nominal",
            "title": "port",
            "sort": "-x"
        },
        "color": {
            "type": "nominal",
            "field": "pointcall_action"
        }
    }
}, pds.DataFrame(not_local_pointcalls))

## Distribution des destinations

In [ ]:
VegaLite({
    "title": "Destinations des bateaux allant à l'extérieur de la région (pointcalls In hors région)",
    "width": 800,
    "mark": "bar",
    "encoding": {
        "x": {
            "type": "quantitative", 
            "aggregate": "count",
            "title": "nombre de pointcalls"
        },
        "y": {
            "field": "pointcall",
            "type": "nominal",
            "title": "port",
            "sort": "-x"
        },
        "color": {
            "type": "nominal",
            "field": "pointcall_uncertainity",
            "title": "incertitude ('pointcall_uncertainty')"
        }
    }
}, pds.DataFrame([p for p in not_local_pointcalls if p["pointcall_action"] == 'in']))

In [ ]:
VegaLite({
    "title": "Nombre de pointcalls pour des bateaux venant depuis l'extérieur de la région",
    "width": 800,
    "mark": "bar",
    "encoding": {
        "x": {
            "type": "quantitative", 
            "aggregate": "count",
            "title": "nombre de pointcalls"
        },
        "y": {
            "field": "pointcall",
            "type": "nominal",
            "title": "port",
            "sort": "-x"
        },
        "color": {
            "type": "nominal",
            "field": "pointcall_uncertainity",
            "title": "Incertitude"
        }
    }
}, pds.DataFrame([p for p in not_local_pointcalls if p["pointcall_action"] == 'out']))

In [ ]:
VegaLite({
    "title": "Destinations des bateaux allant à l'extérieur de la région (pointcalls 'In' hors région)",
    "width": 800,
    "mark": "bar",
    "encoding": {
        "x": {
            "type": "quantitative", 
            "aggregate": "count",
            "title": "nombre de pointcalls"
        },
        "y": {
            "field": "pointcall_as_toflit_partner",
            "type": "nominal",
            "title": "pointcalls par partenaire toflit grouping",
            "sort": "-x"
        },
        "color": {
            "type": "nominal",
            "field": "pointcall_uncertainity",
            "title": "Incertitude"
        }
    }
}, pds.DataFrame([p for p in get_pointcalls_port_as_toflit_partner(not_local_pointcalls, partner_classification='partner_grouping') if p["pointcall_action"] == 'in']))

In [ ]:
VegaLite({
    "title": "Nombre de pointcalls pour des bateaux venant depuis l'extérieur de la région (pointcalls 'Out' hors région)",
    "width": 800,
    "mark": "bar",
    "encoding": {
        "x": {
            "type": "quantitative", 
            "aggregate": "count",
            "title": "nombre de pointcalls"
        },
        "y": {
            "field": "pointcall_as_toflit_partner",
            "type": "nominal",
            "title": "pointcalls par partenaire toflit grouping",
            "sort": "-x"
        },
        "color": {
            "type": "nominal",
            "field": "pointcall_uncertainity",
            "title": "Incertitude ('pointcall_uncertainity')"
        }
    }
}, pds.DataFrame([p for p in get_pointcalls_port_as_toflit_partner(not_local_pointcalls, partner_classification='partner_grouping') if p["pointcall_action"] == 'out']))

## Premier aperçu des produits/objets de voyage ('commodity_purpose')

In [ ]:
# todo affiner quand on aura commodity_action
local_in = [pointcall for pointcall in get_pointcalls_commodity_purposes_as_toflit_product(local_pointcalls, "product_revolutionempire") if p["pointcall_action"] == 'in']
purposes_in = []
for pointcall in local_in:
    purposes_in += [{"port": pointcall["pointcall"], "uncertainty": pointcall["pointcall_uncertainity"], **p} for p in pointcall['commodity_purposes']]

VegaLite({
    "title": "Commodity_purposes (produits ou activités) arrivant dans la région (classification révolution & empire)",
    "width": 600,
    "mark": "bar",
    "encoding": {
        "y": {
            "type": "nominal",
            "field": "commodity_as_toflit",
            "title": "commodity purposes agrégés",
            "sort": "-x"
        },
        "x": {
            "type": "quantitative", 
            "aggregate": "count",
            "title": "nombre de commodity purposes"
        },
        "color": {
            "type": "nominal",
            "field": "uncertainty",
            "title": "incertitude"
        }
    }
}, pds.DataFrame(purposes_in))


In [ ]:
# todo affiner quand on aura commodity_action
local_in = [pointcall for pointcall in get_pointcalls_commodity_purposes_as_toflit_product(local_pointcalls, "product_revolutionempire") if p["pointcall_action"] == 'in']
purposes_in = []
for pointcall in local_in:
    purposes_in += [{"port": pointcall["pointcall"], "admiralty": pointcall["pointcall_admiralty"], **p} for p in pointcall['commodity_purposes']]

VegaLite({
    "title": "Nombre de commodity_purposes (produits ou activités) arrivant dans la région, par ports (classification révolution & empire)",
    "width": 600,
    "mark": "circle",
    "encoding": {
        "x": {
            "type": "nominal",
            "field": "port",
            "sort": "-size"
        },
        "size": {
            "type": "quantitative", 
            "aggregate": "count",
            "title": "nombre de commodity purposes"
        },
        "y": {
            "field": "commodity_as_toflit",
            "type": "nominal",
            "title": "produit",
            "sort": "-size"
        }
    }
}, pds.DataFrame(purposes_in))


In [ ]:
# todo affiner quand on aura commodity_action
local_out = [pointcall for pointcall in get_pointcalls_commodity_purposes_as_toflit_product(local_pointcalls, "product_revolutionempire") if pointcall['pointcall_action'] == 'out']
purposes_out = []
for pointcall in local_out:
    purposes_out += [{"port": pointcall["pointcall"], "uncertainty": pointcall["pointcall_uncertainity"], **p} for p in pointcall['commodity_purposes']]


VegaLite({
"title": "Commodity_purposes (produits ou activités) sortant de la région (classification révolution & empire)",
"width": 600,
"mark": "bar",
"encoding": {
    "y": {
        "type": "nominal",
        "field": "commodity_as_toflit",
        "title": "commodity purposes agrégés",
        "sort": "-x"
    },
    "x": {
        "type": "quantitative", 
        "aggregate": "count",
        "title": "nombre de commodity purposes"
    },
    "color": {
        "type": "nominal",
        "field": "uncertainty",
        "title": "Incertitude"
    }
}
}, pds.DataFrame(purposes_out))


In [ ]:
VegaLite({
    "title": "Nombre de commodity_purposes (produits ou activités) sortant de la région, par ports (classification révolution & empire)",
    "width": 600,
    "mark": "circle",
    "encoding": {
        "x": {
            "type": "nominal",
            "field": "port",
            "sort": "-size"
        },
        "size": {
            "type": "quantitative", 
            "aggregate": "count",
            "title": "nombre de commodity purposes"
        },
        "y": {
            "field": "commodity_as_toflit",
            "type": "nominal",
            "title": "commodity purposes (agrégés)",
            "sort": "-size"
        }
    }
}, pds.DataFrame(purposes_out))

In [ ]:
# todo affiner quand on aura commodity_action
global_in = [pointcall for pointcall in get_pointcalls_commodity_purposes_as_toflit_product(not_local_pointcalls, "product_revolutionempire") if pointcall['pointcall_action'] == 'in']
purposes_in = []
for pointcall in global_in:
    purposes_in += [{"port": pointcall["pointcall"], **p} for p in pointcall['commodity_purposes']]


VegaLite({
    "title": "Nombre de commodity_purposes (produits ou activités) envoyés depuis la région vers l'extérieur, par ports (classification révolution & empire)",
    "width": 1500,
    "mark": "circle",
    "encoding": {
        "x": {
            "type": "nominal",
            "field": "port",
            "sort": "-size"
        },
        "size": {
            "type": "quantitative", 
            "aggregate": "count",
            "title": "nombre de commodity purposes"
        },
        "y": {
            "field": "commodity_as_toflit",
            "type": "nominal",
            "title": "produit",
            "sort": "-size"
        }
    }
}, pds.DataFrame(purposes_in))

In [ ]:
# todo affiner quand on aura commodity_action
global_out = [pointcall for pointcall in get_pointcalls_commodity_purposes_as_toflit_product(not_local_pointcalls, "product_revolutionempire") if pointcall['pointcall_action'] == 'out']
purposes_out = []
for pointcall in global_out:
    purposes_out += [{"port": pointcall["pointcall"], "admiralty": pointcall["pointcall_admiralty"], **p} for p in pointcall['commodity_purposes']]


VegaLite({
    "title": "Nombre de commodity_purposes (produits ou activités) arrivant dans la région depuis l'extérieur, par ports (classification révolution & empire)",
    "width": 600,
    "mark": "circle",
    "encoding": {
        "x": {
            "type": "nominal",
            "field": "port",
            "sort": "-size"
        },
        "size": {
            "type": "quantitative", 
            "aggregate": "count",
            "title": "nombre de commodity purposes"
        },
        "y": {
            "field": "commodity_as_toflit",
            "type": "nominal",
            "title": "produit",
            "sort": "-size"
        }
    }
}, pds.DataFrame(purposes_out))


## Quels sont les produits en commun avec toflit18 ?


In [ ]:
from poitousprint import Toflit, get_pointcalls_commodity_purposes_as_toflit_product

toflit_client = Toflit()

"""
Objectif : 
Pour les produit commercés / transportés pour les flux concernant le datasprint, obtenir 3 ensembles :
- ensemble 1 - produits qu'on retrouve à la fois dans Toflit18 et Portic
- ensemble 2 - produits qu'on retrouve exclusivement dans Toflit18
- ensemble 3 - produits qu'on retrouve exclusivement dans Portic

from poitousprint import Portic, Toflit, get_pointcalls_commodity_purposes_as_toflit_product

toflit_client = Toflit()
portic_client = Portic()
"""
print('')

In [ ]:
# choix d'une classification d'alignement
chosen_classification = 'product_revolutionempire'
# exemples d'autres classifications possibles : product_source', 'product_orthographic', 'product_simplification'


In [ ]:
# récupération des pointcalls Portic qui concernent le datasprint
pointcalls_datasprint = portic_client.get_pointcalls(
 source_subset = 'Poitou_1789'
)

# enrichissement de ces pointcalls avec la propriété 'commodity_as_toflit' qui nous donne l'équivalent des produits navigo dans Toflit18 avec la classification choisie
croisement_produits = get_pointcalls_commodity_purposes_as_toflit_product(pointcalls_datasprint, product_classification=chosen_classification)

In [ ]:
navigo_products_for_datasprint = set()
for pointcall in croisement_produits:
    if pointcall['commodity_purposes'] is not None:
        for commodity in pointcall['commodity_purposes']:
            navigo_products_for_datasprint.add(commodity['commodity_as_toflit'])

# navigo_products_for_datasprint
print('')

In [ ]:
# récupération des flows Toflit18 qui concernent le datasprint
flows = toflit_client.get_flows(
    year=1789, 
    customs_region='La Rochelle',
)

toflit_products_for_datasprint = set()
for flow in flows:
    toflit_products_for_datasprint.add(flow[chosen_classification])

# toflit_products_for_datasprint
print('')

In [ ]:
result1 = toflit_products_for_datasprint.intersection(navigo_products_for_datasprint)

result2 = toflit_products_for_datasprint.difference(navigo_products_for_datasprint)

result3 = navigo_products_for_datasprint.difference(toflit_products_for_datasprint)

def format_p(s):
    l = list(s)
    l = [i for i in l if i is not None]
    l.sort()
    print ('\n'.join(l))
    

print("Ensemble 1 - produits qu'on retrouve à la fois dans Toflit18 et Portic :\n\n")
format_p(result1)
print("\n\nEnsemble 2 - produits qu'on retrouve exclusivement dans Toflit18 : \n\n")
format_p(result2)
print("\n\nEnsemble 3 - produits qu'on retrouve exclusivement dans Portic : \n\n")
format_p(result3)

## Correspondance entre les ports et les bureaux de ferme



In [ ]:
"""
### Objectif : Représenter l'intersection et l'exclusion des ports Navigo avec les bureaux des Fermes Toflit

- grosseur des liens dépend du nombre de pointcalls concernés
- colorer de la même manière tous les ports rattachés à un même bureau des Fermes

=> diagramme alluvial avec RAWGraphs
"""
# récupération de tous les ports navigo concernés

pointcalls_datasprint = portic_client.get_pointcalls(
 year = 1789,
 pointcall_admiralty = ['La Rochelle', 'Marennes', 'Sables d\'Olonne'],
 # si on utilise source_subset = 'Poitou_1789' on aura des ports qui ne correspondent pas à la direction des Fermes de La Rochelle => trop de ports (invisualisable)
 params = ['pointcall', 'ferme_bureau', 'pointcall_admiralty', 'source_subset']
)

In [ ]:
# écriture des croisements dans un csv pour nourrir RAWGraphs
import csv
from csv import DictWriter

# tableau qui contient une ligne par pointcall (avec à chaque fois le port Navigo qui correspond à un bureau Toflit) 
with open('dumps/croisement_ports_bureaux_datasprint.csv', 'w', newline='') as csvfile:
        fieldnames = ['port_Navigo','ferme_bureau_Toflit']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for pointcall in pointcalls_datasprint:
            
            # remplacer par "pas de bureau rattaché" plutot que None quand le port n'a pas de bureau des Fermes associé
            bureau = pointcall['ferme_bureau']
            if bureau is None:
                bureau = "pas de bureau rattaché"
            writer.writerow({
                'port_Navigo':pointcall['pointcall'],
                'ferme_bureau_Toflit':bureau
            })

Voici un tableau (au format csv) disponible sur le répertoire github qui correspond aux croisements entre les ports et les bureaux de ferme : https://github.com/medialab/portic-datasprint-2021/blob/main/preliminary_inquiry/dumps/croisement_ports_bureaux_datasprint.csv

Ce csv est exploitable pour customiser / refaire une visualisation

In [ ]:
from IPython.core.display import SVG
SVG(filename='dumps/visualisations/croisement ports Portic : bureaux des Fermes Navigo - datasprint PORTIC 2021/diagramme_alluvial_ports_bureaux.svg')

#### Légende

À gauche : les bureaux des Fermes Toflit

À droite : les ports Navigo qui s'y rattachent

## Correspondance entre les ports non-français et les partenaires commerciaux tels que décrits dans Toflit18


In [ ]:
"""
### Objectif : obtenir un diagramme alluvial qui représente le croisement entre ports Navigo dans lesquels on a des pointcalls 'In' (ce qu'on pourrait analyser comme des destinataires d'exports) et les partenaires commerciaux Toflit18 pour les flux concernant le sprint

=> RAWGraphs
"""

chosen_pointcall_actions = 'In'

In [ ]:
# récupération de tous les ports navigo concernés

pointcalls_datasprint = portic_client.get_pointcalls(
 source_subset = 'Poitou_1789',
 pointcall_action = chosen_pointcall_actions
)

croisement_partners = get_pointcalls_port_as_toflit_partner(pointcalls_datasprint, 'partner_grouping')

In [ ]:
# écriture des alignements dans un dict avec une ligne par match pour grossir les liens selon quantité de pointcalls concernés dans le diagramme alluvial
ports_navigo = []

for pointcall in croisement_partners: 
    # remplacer par "pas de partenaire commercial rattaché" plutot que None quand le port Navigo ne correspond pas à un partenaire Toflit18
    partner = pointcall['pointcall_as_toflit_partner']
    if partner is None:
        partner = "pas de partenaire commercial rattaché"
        
    ports_navigo.append({
        'port_navigo': pointcall['pointcall'],
        'toflit_partner':partner
    })


In [ ]:
# écriture dans un csv
import csv
from csv import DictWriter

# tableau qui contient une colonne par clé de dict 
with open('dumps/croisement_ports_' + str(chosen_pointcall_actions) + '_partners_datasprint.csv', 'w', newline='') as csvfile:
        fieldnames = ['port_navigo','toflit_partner']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for pointcall in ports_navigo:
            writer.writerow(pointcall)

Voici un tableau (au format csv - disponible sur le répertoire github) qui présente sous une forme simplifiée les relations entre ports de départ et partenaires étrangers (déduits des ports d'arrivée) : https://github.com/medialab/portic-datasprint-2021/blob/main/preliminary_inquiry/dumps/croisement_ports_In_partners_datasprint.csv

Ce csv est exploitable pour customiser / refaire une visualisation.

In [ ]:
from IPython.core.display import SVG
SVG(filename='dumps/visualisations/croisement ports Portic : bureaux des Fermes Navigo - datasprint PORTIC 2021/diagramme_alluvial_ports_partners.svg')



#### Légende

À gauche : les partenaires Toflit

À droite : les ports Navigo (destinataires d'exports) qui s'y rattachent

#### Remarque

Ici, les ports navigo considérés destinataires d'exports sont ceux associés avec un pointcall "In" faisant partie des pointcalls du sprint.

## Premier aperçu des réseaux de départs/destinations dans navigo


In [ ]:
"""
#### Objectifs :
- réseaux monopartites entre ports : voir quels ports échangent beaucoup entre eux
- réseaux bipartites departs / destinations : voir dans quel sens se passent les flux

Remarques :
- Les couleurs des réseaux bipartites mettent en valeur les ports ou autres entités administratives concernés par le sprint (couleurs foncées)
- on peut utiliser une fonction qui permet filtrer les données avant la construction d'un réseau bipartite et ne construire que les noeuds et liens du réseau qui concernent le sprint

=> ces fonctions de coloration de de filtre peuvent être facilement mises en place pour les réseaux monopartites
"""

from poitousprint import Portic, Toflit, get_pointcalls_port_as_toflit_partner, get_flows_or_travels_port_as_toflit_partner
import json
import networkx as nx
from ipysigma import Sigma

portic_client = Portic()
toflit_client = Toflit()

# this function allows to map a value from a domain of min-max to another
def map_value(value, domain_min, domain_max, range_min, range_max):
    left_span = domain_max - domain_min
    right_span = range_max - range_min

    # Convert the left range into a 0-1 range (float)
    scaled = float(value - domain_min) / float(left_span)

    # Convert the 0-1 range into a value in the right range.
    return range_min + (scaled * right_span)

In [ ]:
"""
#### Fonctions génériques qui permettent de fabriquer des réseaux monopartites et bipartites à partir :

1. d'une liste de dicts (ex. flux toflit18)
2. de propriétés des dicts (1 pour les réseaux monopartites, 2 pour les bipartites)
"""
def render_monopartite_graph_port_exchanges(data, key_1, key_2, params=None): # data could be Navigo flows or travels 

    # créer un graphe
    Graph = nx.Graph()

    # créer des dict pour l'unique type de noeuds et les liens
    key_uniq = {}
    edges_uniq = {}
    default_params = {
        "color_1": "rgb(0, 255, 0)", # noeuds de type 1 en vert
        "color_2": "rgb(255, 0, 0)", # noeuds de type 2 en rouge
        "node_min_size": 1,
        "node_max_size": 10
    }
    final_params = default_params
    if params is not None :
        final_params = {
            *default_params,
            *params
        }
        
    # remplir les dicts
    for datum in data:
        if key_1 in datum and key_2 in datum:
            value_1 = datum[key_1] if datum[key_1] is not None else "undefined"
            value_2 = datum[key_2] if datum[key_2] is not None else "undefined"
            value_1_id = value_1
            value_2_id = value_2

            if value_1_id in key_uniq:
                key_uniq[value_1_id] = {**key_uniq[value_1_id], "size": key_uniq[value_1_id]["size"] + 1}
            else:
               key_uniq[value_1_id] = {
                   "type": key_1, 
                   "label": value_1, 
                   "color": "rgb(0, 255, 0)",
                   "size": 1
               }

            if value_2_id in key_uniq:
                key_uniq[value_2_id] = {**key_uniq[value_2_id], "size": key_uniq[value_2_id]["size"] + 1}
            else:
               key_uniq[value_2_id] = {
                   "type": key_2, 
                   "label": value_2, 
                   "color": "rgb(0, 255, 0)",
                   "size": 1
               }

            edge_footprint = value_1_id + "-" + value_2_id

            if edge_footprint in edges_uniq:
                edges_uniq[edge_footprint]["weight"] += 1
            else:
                edges_uniq[edge_footprint] = {
                    "source": value_1_id,
                    "target": value_2_id,
                    "weight": 1
                }
    
    
    # concaténer les deux dicts de noeuds en un seul => obsolète dans ce cas
    all_nodes = key_uniq
    # all_nodes.update(# autre dict de noeuds )

    # applatir et formatter les noeuds
    nodes = []
    for key, node in all_nodes.items():
        nodes.append((key, node))
    edges = []

    for key, edge in edges_uniq.items():
        edges.append((edge["source"], edge["target"], {"weight": edge["weight"]}))

    domain_min_nodes_size = min([node[1]['size'] for node in nodes])
    domain_max_nodes_size = max([node[1]['size'] for node in nodes])
    range_in_nodes_size = [1, 10]
    nodes_size_mapping_params = [domain_min_nodes_size, domain_max_nodes_size, *range_in_nodes_size]

    for node in nodes:
        node[1]["size"] = map_value(node[1]["size"], *nodes_size_mapping_params)
        # node[1]["label"] = node[1]["name"]
    
    Graph.add_nodes_from(nodes)
    Graph.add_edges_from(edges)

    return Sigma(Graph, start_layout=True)

In [ ]:
def render_coocurrences_graph_sprint_colored(data, key_1, key_2, params=None):
    # créer un graphe
    Graph = nx.Graph()

    # créer des dict pour les deux types de noeuds et les liens
    key1_uniq = {}
    key2_uniq = {}
    edges_uniq = {}
    default_params = {
        "color_1": "rgb(51, 204, 255)", # noeuds de type 1 en bleu clair
        "color_2": "rgb(238, 130, 238)", # noeuds de type 2 en rose
        "node_min_size": 1,
        "node_max_size": 10
    }
    final_params = default_params
    if params is not None :
        final_params = {
            *default_params,
            *params
        }
        
    # colorer les ports / ou autres entités rattachées à la direction des fermes de La Rochelle (au sprint)
    # en bleu si entité de départ
    # en violet si entité d'arrivée
    sprint_ughs_id = ['A1964694','A0171758','A0136930','A0196496','A0198999','A0137148','A0127055','A0133403','A0213721','A0199508','A0148208','A0141325','A0138533','A1964982','A0186515','A0124809','A1964767','A0172590','A0181608','A0165077','A0169240','A0165056','A1963997','A0136403','A0195938','A0122971','A0207992']
    sprint_bureaux = ['Alligre', 'Saint-Martin île de Ré', 'Marennes', 'Rochefort', 'Charente', 'La Rochelle', 'Sables-d\'Olonne']
    sprint_direction = 'La Rochelle'
    
    # choisir par rapport à quelles valeurs on va filtrer la coloration bleue, en fonction des keys données en argument
    chosen_filter1 = None
    if key_1 == 'departure_fr':
        chosen_filter1 = sprint_ughs_id
    elif key_1 == 'departure_ferme_bureau':
        chosen_filter1 = sprint_bureaux
    elif key_1 == 'departure_ferme_direction':
        chosen_filter1 = sprint_direction 
    
    # choisir par rapport à quelles valeurs on va filtrer la coloration violette, en fonction des keys données en argument
    chosen_filter2 = None
    if key_2 == 'destination_fr':
        chosen_filter2 = sprint_ughs_id
    elif key_2 == 'destination_ferme_bureau':
        chosen_filter2 = sprint_bureaux
    elif key_2 == 'destination_ferme_direction':
        chosen_filter2 = sprint_direction 
    
    # remplir les dicts
    for datum in data:
    
        if key_1 in datum and key_2 in datum:
            value_1 = datum[key_1] if datum[key_1] is not None else "undefined"
            value_2 = datum[key_2] if datum[key_2] is not None else "undefined"
            value_1_id = key_1 + "_" + value_1
            value_2_id = key_2 + "_" + value_2
            
            # on changera la variable color attribuée au noeud s'il passe le filtre de rattachement au sprint
            color = final_params["color_1"]
        
            if value_1_id in key1_uniq:
                key1_uniq[value_1_id] = {**key1_uniq[value_1_id], "size": key1_uniq[value_1_id]["size"] + 1}
            else:
                # coloration en bleu si le noeud se rattache à la direction des fermes de La Rochelle
                if (chosen_filter1 == sprint_ughs_id and datum['departure_uhgs_id'] is not None and datum['departure_uhgs_id'] in sprint_ughs_id):
                    color = "rgb (0, 0, 255)"
                elif (chosen_filter1 == sprint_bureaux and datum['departure_ferme_bureau'] is not None and datum['departure_ferme_bureau'] in sprint_bureaux):
                    color = "rgb (0, 0, 255)"
                elif (chosen_filter1 == sprint_direction and datum['departure_ferme_direction'] is not None and datum['departure_ferme_direction'] in sprint_direction) :
                    color = "rgb (0, 0, 255)"
                    
                key1_uniq[value_1_id] = {
                   "type": key_1, 
                   "name": value_1, 
                   "color": color,
                   "size": 1
                }
            
            # on changera la variable color attribuée au noeud s'il passe le filtre de rattachement au sprint
            color = final_params["color_2"]
            
            if value_2_id in key2_uniq:
                key2_uniq[value_2_id] = {**key2_uniq[value_2_id], "size": key2_uniq[value_2_id]["size"] + 1}
            else:
                # coloration en violet si le noeud se rattache à la direction des fermes de La Rochelle
                if (chosen_filter2 == sprint_ughs_id and datum['destination_uhgs_id'] is not None and datum['destination_uhgs_id'] in sprint_ughs_id):
                    color = "rgb(70, 0, 128)"
                elif (chosen_filter2 == sprint_bureaux and datum['destination_ferme_bureau'] is not None and datum['destination_ferme_bureau'] in sprint_bureaux):
                    color = "rgb(70, 0, 128)"
                elif (chosen_filter2 == sprint_direction and datum['destination_ferme_direction'] is not None and datum['destination_ferme_direction'] in sprint_direction) :
                    color = "rgb(70, 0, 128)"
                    
                key2_uniq[value_2_id] = {
                   "type": key_2, 
                   "name": value_2, 
                   "color": color,
                   "size": 1
               }
                
            edge_footprint = value_1_id + "-" + value_2_id
            if edge_footprint in edges_uniq:
                edges_uniq[edge_footprint]["weight"] += 1
            else:
                edges_uniq[edge_footprint] = {
                    "source": value_1_id,
                    "target": value_2_id,
                    "weight": 1
                }
    # concaténer les deux dicts de noeuds en un seul
    all_nodes = key1_uniq
    all_nodes.update(key2_uniq)
    # applatir et formatter les noeuds
    nodes = []
    for key, node in all_nodes.items():
        nodes.append((key, node))
    edges = []

    for key, edge in edges_uniq.items():
        edges.append((edge["source"], edge["target"], {"weight": edge["weight"]}))
        
    # ajuster la taille des noeuds en fonction d'un min et d'un max donnés
    domain_min_nodes_size = min([node[1]['size'] for node in nodes])
    domain_max_nodes_size = max([node[1]['size'] for node in nodes])
    range_in_nodes_size = [final_params["node_min_size"], final_params["node_max_size"]]
    nodes_size_mapping_params = [domain_min_nodes_size, domain_max_nodes_size, *range_in_nodes_size]

    for node in nodes:
        node[1]["size"] = map_value(node[1]["size"], *nodes_size_mapping_params)
        node[1]["label"] = node[1]["name"]


    Graph.add_nodes_from(nodes)
    Graph.add_edges_from(edges)

    return Sigma(Graph, start_layout=True)

In [ ]:
def render_coocurrences_graph_sprint_filtered(data, key_1, key_2, params=None):
    
    # créer un graphe
    Graph = nx.Graph()

    # créer des dict pour les deux types de noeuds et les liens
    key1_uniq = {}
    key2_uniq = {}
    edges_uniq = {}
    default_params = {
        "color_1": "rgb(0, 0, 255)", # noeuds de type 1 en bleu foncé
        "color_2": "rgb(70, 0, 128)",# noeuds de type 2 en violet foncé 
        "node_min_size": 1,
        "node_max_size": 10
    }
    final_params = default_params
    if params is not None :
        final_params = {
            *default_params,
            *params
        }
        
    # filtrer les ports / ou autres entités rattachées à la direction des fermes de La Rochelle (au sprint)
    sprint_ughs_id = ['A1964694','A0171758','A0136930','A0196496','A0198999','A0137148','A0127055','A0133403','A0213721','A0199508','A0148208','A0141325','A0138533','A1964982','A0186515','A0124809','A1964767','A0172590','A0181608','A0165077','A0169240','A0165056','A1963997','A0136403','A0195938','A0122971','A0207992']
    sprint_bureaux = ['Alligre', 'Saint-Martin île de Ré', 'Marennes', 'Rochefort', 'Charente', 'La Rochelle', 'Sables-d\'Olonne']
    sprint_direction = 'La Rochelle'
    
    # choisir par rapport à quelles valeurs on va filtrer, en fonction des keys données en argument
    chosen_filter1 = None
    if key_1 == 'departure_fr':
        chosen_filter1 = sprint_ughs_id
    elif key_1 == 'departure_ferme_bureau':
        chosen_filter1 = sprint_bureaux
    elif key_1 == 'departure_ferme_direction':
        chosen_filter1 = sprint_direction 
    
    # choisir par rapport à quelles valeurs on va filtrer la coloration violette, en fonction des keys données en argument
    chosen_filter2 = None
    if key_2 == 'destination_fr':
        chosen_filter2 = sprint_ughs_id
    elif key_2 == 'destination_ferme_bureau':
        chosen_filter2 = sprint_bureaux
    elif key_2 == 'destination_ferme_direction':
        chosen_filter2 = sprint_direction 
    
    # remplir les dicts
    for datum in data:
    
        if key_1 in datum and key_2 in datum:
            value_1 = datum[key_1] if datum[key_1] is not None else "undefined"
            value_2 = datum[key_2] if datum[key_2] is not None else "undefined"
            value_1_id = key_1 + "_" + value_1
            value_2_id = key_2 + "_" + value_2
            
            # on n'ajoutera les noeuds et le lien entre eux que s'ils passent le filtre de rattachement au sprint
            valid_node_1 = False
            valid_node_2 = False
            if (chosen_filter1 == sprint_ughs_id and datum['departure_uhgs_id'] is not None and datum['departure_uhgs_id'] in sprint_ughs_id):
                valid_node_1 = True
            elif (chosen_filter1 == sprint_bureaux and datum['departure_ferme_bureau'] is not None and datum['departure_ferme_bureau'] in sprint_bureaux):
                valid_node_1 = True
            elif (chosen_filter1 == sprint_direction and datum['departure_ferme_direction'] is not None and datum['departure_ferme_direction'] in sprint_direction) :
                valid_node_1 = True
            if (chosen_filter2 == sprint_ughs_id and datum['destination_uhgs_id'] is not None and datum['destination_uhgs_id'] in sprint_ughs_id):
                valid_node_2 = True
            elif (chosen_filter2 == sprint_bureaux and datum['destination_ferme_bureau'] is not None and datum['destination_ferme_bureau'] in sprint_bureaux):
                valid_node_2 = True
            elif (chosen_filter2 == sprint_direction and datum['destination_ferme_direction'] is not None and datum['destination_ferme_direction'] in sprint_direction) :
                valid_node_2 = True
            
            if valid_node_1 and valid_node_2:
                
                if value_1_id in key1_uniq:
                    key1_uniq[value_1_id] = {**key1_uniq[value_1_id], "size": key1_uniq[value_1_id]["size"] + 1}
                else:
                    key1_uniq[value_1_id] = {
                       "type": key_1, 
                       "name": value_1, 
                       "color": final_params['color_1'],
                       "size": 1
                    }

                if value_2_id in key2_uniq:
                    key2_uniq[value_2_id] = {**key2_uniq[value_2_id], "size": key2_uniq[value_2_id]["size"] + 1}
                else:
                    key2_uniq[value_2_id] = {
                       "type": key_2, 
                       "name": value_2, 
                       "color": final_params['color_2'],
                       "size": 1
                    }

                edge_footprint = value_1_id + "-" + value_2_id
                if edge_footprint in edges_uniq:
                    edges_uniq[edge_footprint]["weight"] += 1
                else:
                    edges_uniq[edge_footprint] = {
                        "source": value_1_id,
                        "target": value_2_id,
                        "weight": 1
                    }
                    
    # concaténer les deux dicts de noeuds en un seul
    all_nodes = key1_uniq
    all_nodes.update(key2_uniq)
    # applatir et formatter les noeuds
    nodes = []
    for key, node in all_nodes.items():
        nodes.append((key, node))
    edges = []

    for key, edge in edges_uniq.items():
        edges.append((edge["source"], edge["target"], {"weight": edge["weight"]}))
        
    # ajuster la taille des noeuds en fonction d'un min et d'un max donnés
    domain_min_nodes_size = min([node[1]['size'] for node in nodes])
    domain_max_nodes_size = max([node[1]['size'] for node in nodes])
    range_in_nodes_size = [final_params["node_min_size"], final_params["node_max_size"]]
    nodes_size_mapping_params = [domain_min_nodes_size, domain_max_nodes_size, *range_in_nodes_size]

    for node in nodes:
        node[1]["size"] = map_value(node[1]["size"], *nodes_size_mapping_params)
        node[1]["label"] = node[1]["name"]


    Graph.add_nodes_from(nodes)
    Graph.add_edges_from(edges)

    return Sigma(Graph, start_layout=True)

In [ ]:
# 0. Je vais chercher les pointcalls / flows / travels du sprint (côté Navigo) pour nourrir mes réseaux
pointcalls_datasprint = portic_client.get_pointcalls(
    source_subset = 'Poitou_1789')

print("Nombre de pointcalls trouvés concernant le sprint : ", len(pointcalls_datasprint))

pointcalls_with_toflit_partners = get_pointcalls_port_as_toflit_partner(pointcalls_datasprint, 'partner_grouping')

In [ ]:
flows_datasprint = portic_client.get_flows(
        ports = ['A1964694', # je fonctionne avec tous les UHGS_id des ports associés à la direction des fermes de La Rochelle (voir fichier aligenement localités dans le drive : Noirmoutier et Ile de Bouin sont pris en compte)
                'A0171758',
                'A0136930',
                'A0196496',
                'A0198999',
                'A0137148',
                'A0127055',
                'A0133403',
                'A0213721',
                'A0199508',
                'A0148208',
                'A0141325',
                'A0138533',
                'A1964982',
                'A0186515',
                'A0124809',
                'A1964767',
                'A0172590',
                'A0181608',
                'A0165077',
                'A0169240',
                'A0165056',
                'A1963997',
                'A0136403',
                'A0195938',
                'A0122971',
                'A0207992'], # sinon on devrait pouvoir fonctionner avec source_subset = 'Poitou_1789' mais pour l'instant ne fonctionne pas pour moi
    year = 1789)

print("Nombre de flows trouvés concernant le sprint : ", len(flows_datasprint))

# cette fonction ajoute les attributs "departure_fr_as_toflit_partner" et "destination_fr_as_toflit_partner" à un flow ou travel navigo (à perfectionner => pour l'instant elle n'associe pas de partenaire aux ports Etrangers du sprint, et ne semble fonctionner que pour la France ...)
flows_with_toflit_partners = get_flows_or_travels_port_as_toflit_partner(flows_datasprint, 'partner_grouping')

In [ ]:
# note : #### Les travels peuvent être utilisés de manière équivalente aux flows pour nourrir les réseaux si l'on veut comparer les données disponibles en prenant flows ou travels dans l'API Navigo (à priori plus de données avec flows)
travels_datasprint = portic_client.get_travels(source_subset = 'Poitou_1789')

print("Nombre de travels trouvés concernant le sprint : ", len(travels_datasprint))
travels_with_toflit_partners = get_flows_or_travels_port_as_toflit_partner(travels_datasprint, 'partner_grouping')

###  Réseaux des voyages entre ports de la région

In [ ]:
"""
## 1. Fabriction de réseau monopartite : échanges entre ports

#### Si 2 ports sont dans le même flow ou travel, ils sont reliés. Plus les échanges sont intenses, plus les ports sont proches sur le graphe
"""
render_monopartite_graph_port_exchanges(flows_datasprint, 'departure_fr', 'destination_fr')

<img src="dumps/visualisations/navigo/navigo_monopartite_1.png">



## Réseaux des voyages entre ports de la région rapportés à la classification des partenaires de toflit18

In [ ]:
render_monopartite_graph_port_exchanges(flows_with_toflit_partners, 'departure_fr_as_toflit_partner', 'destination_fr_as_toflit_partner')

<img src="dumps/visualisations/navigo/navigo_monopartite_2.png">


## 2. Fabrication de réseaux bipartites : départ / arrivée

#### Si 2 ports font partie du même flow, ou du même travel, ils seront reliés sur le graphe



Remarques : 
- Peut-être intéressant de filtrer les ports de départ / d'arrivée sur la direction des Fermes de La Rochelle (pour savoir où vont les bateaux qui partent de la région / d'où viennent les bateaux arrivent dans la région)

### Réseau bipartite des destinations et des arrivées pour les données du sprint

In [ ]:
render_coocurrences_graph_sprint_colored(flows_datasprint, "departure_fr", "destination_fr")

<img src="dumps/visualisations/navigo/navigo_monopartite_3.png">


#### Légende

Départ en bleu - arrivée en violet (ports de la région) ou rose (ports hors région)

#### Remarques

Certains ports apparaissent deux fois dans cette visualisation car ils sont à la fois des ports de départ et de destination.



###  Réseau bipartite des départs et arrivés, regroupés par partenaires toflit18

In [ ]:
render_coocurrences_graph_sprint_colored(flows_with_toflit_partners, "departure_fr_as_toflit_partner", "destination_fr")

<img src="dumps/visualisations/navigo/navigo_monopartite_4.png">

#### Légende

- ports de départ en bleu clair / bleu foncé si fait partie du sprint
- ports d'arrivée en rose / violet si fait partie du sprint

### Réseau bipartite des arrivés et départs, regroupés par bureau des Fermes de départ / partenaire Toflit18 d'arrivée


In [ ]:
"""
### D. croisement (visualisation de données Navigo avec des entités Toflit) : bureau des Fermes Toflit de départ / partenaire Toflit d'arrivée
"""
render_coocurrences_graph_sprint_colored(flows_with_toflit_partners, "departure_ferme_bureau", "destination_fr_as_toflit_partner")

<img src="dumps/visualisations/navigo/navigo_monopartite_6.png">

#### Légende

Départs en bleu

Arrivées en rose/violet

